# SeokHyeon Chin, 2016320179

## Mini-batch Stochastic Gradient Descent

# Reading Data

In [1]:
import sys
sys.path.insert(0, '..')
%matplotlib inline
import d2l
import torch
import torch.nn as nn
import numpy as np
import time

In [2]:
def get_data():
    data = np.genfromtxt('../data/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    return torch.Tensor(data[:1500, :-1]), torch.Tensor(data[:1500, -1])

features, labels =get_data()
features.shape

torch.Size([1500, 5])

# Implementation from Scratch

In [7]:
def sgd(params, states, hyperparams,batch_size):
    for p in params:
        p.data.sub_(hyperparams['lr'] * p.grad)
        p.grad.data.zero_()

In [8]:
def train(trainer_fn, states, hyperparams, features, labels,batch_size=10, num_epochs=2):
    
    # Initialize model parameters
    net, loss = d2l.linreg, d2l.squared_loss
    w = torch.ones(()).new_empty((features.shape[1], 1),requires_grad=True)
    b= torch.zeros((1,),requires_grad=True)
    l= torch.zeros((1500,1),requires_grad=True)
    def eval_loss():
        return loss(net(features, w, b), labels).mean()
    ls, ts = [eval_loss()], [0,]
    data_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(features, labels), batch_size, shuffle=True)
    start = time.time()
    for _ in range(num_epochs):
        for batch_i, (X, y) in enumerate(data_iter):
            l = loss(torch.mm(X,w)+b,y).mean()
            l.backward()
            trainer_fn([w, b], states, hyperparams,batch_size)
            if (batch_i + 1) * batch_size % 10 == 0:
                ts.append(time.time() - start + ts[-1])
                ls.append(eval_loss())
                start = time.time()
    print('loss: %f, %f sec per epoch' % (ls[-1], ts[-1]/num_epochs))
    d2l.set_figsize()
    d2l.plt.plot(np.linspace(0, num_epochs, len(ls)), ls)
    d2l.plt.xlabel('epoch')
    d2l.plt.ylabel('loss')
    return ts, ls  

In [11]:
def train_sgd(lr, batch_size, num_epochs=2):
    return train(sgd, None, {'lr': lr}, features, labels, batch_size, num_epochs)
gd_res = train_sgd(1, 1500,4)

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

In [10]:
sgd_res = train_sgd(0.005, 1)

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'